# Optimizing Model Parameters

Now that we have a model and data its time to train, validate and test our model by optimizing its parameters on our data. Training a model is an iterative process; in each iteration the model makes a guess about the output, calculates the error in its guess (loss), collects the derivatives of the error with respect to its parameters, an optimizes these parameters using gradient descent

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=False,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=False,
    transform=ToTensor()
)

In [3]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [5]:
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Hyperparameters

Hyperparameters are adustable parameters that let you control the model optimization process.

Differnt hyperparameters values can impact model training and convergence rate. We define the following hyperparameters for training:
- Number of epochs - numbe rof times to iterate over the dataset
- batch size - the number of data samples propagated though the network before the parameters are updated
- Learning rate - how much to update models parameters at each batch/epoch. Smaller values yield slow learning speed, while large value may result in unpredictable behavior training

In [6]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Optimization loop

Once we set our hyperparameters, we can then train and optimize our model with an optimization loop. Each iteration of the optimization loop is called an epoch.

Each epoch consists of two main parts:
- the train loop - iterate over the training dataset and try to converge to optimal parameters.
- the validation / test loop iterate over the test dataset to check if model performance is imporoving

## Loos Function

When presented with some training data, our untrained network is likely not to give the correct answer. Looss function measures the degree of dissimilarity of obtrained result to the target value, and it is the loss function that we want to minimize during training.

To calculate the loss we make a prediction using the inputs of our given data sample and compare it against the true data label value.

In [7]:
loss_fn = nn.CrossEntropyLoss() #note many others like nn.MSELoss, nnNLLLoss, etc

## Optimizer 

Optimization is the process of adjusting model parameters to reduce model error in each training step. Optimization algorithms define how this process is performed. All optimization logic is encapsulated in the optimizer object.

Inside the training loop, optimization happens in three steps:
- call zero_grad to reset the gradients of model parameters. Gradients by default add up; to prevent double counting, we zero them at each iteration
- backpropagate the prediction loss with a call to loss.backward
- Once we have our gradients, we call the optimizer.step to adjust the parameters by the gradients collected in the backward pass

In [8]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## Full Implementation

We define train_loop that loop over our optimization code, ad test_lop that evaluates the model performance against our test data

In [9]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # set the model to training mode 
    # Important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [10]:
def test_loop(dataloader, model, loss_fn):
    # set the model to evaluation mode
    # important for batch normalization and dropout layers
    # Unnecessary in the situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # evaluating the model with torch.no_grad ensures that no gradients are 
    #  computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for 
    #  tensors with requires_grad=true

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):0.1f}%, Avg loss: {test_loss:>8f} \n")

We initialize the loss function and optimizer, and pass it to train loop and test loop.

In [11]:
loos_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------
loss: 2.301270 [   64/60000]
loss: 2.289834 [ 6464/60000]
loss: 2.269948 [12864/60000]
loss: 2.267678 [19264/60000]
loss: 2.248209 [25664/60000]
loss: 2.210048 [32064/60000]
loss: 2.239099 [38464/60000]
loss: 2.193348 [44864/60000]
loss: 2.197321 [51264/60000]
loss: 2.164119 [57664/60000]
Test Error: 
 Accuracy: 28.6%, Avg loss: 2.155802 

Epoch 2
-------------------
loss: 2.161638 [   64/60000]
loss: 2.149521 [ 6464/60000]
loss: 2.093071 [12864/60000]
loss: 2.112159 [19264/60000]
loss: 2.053027 [25664/60000]
loss: 1.995128 [32064/60000]
loss: 2.036783 [38464/60000]
loss: 1.950498 [44864/60000]
loss: 1.965462 [51264/60000]
loss: 1.889827 [57664/60000]
Test Error: 
 Accuracy: 57.1%, Avg loss: 1.883304 

Epoch 3
-------------------
loss: 1.911616 [   64/60000]
loss: 1.876655 [ 6464/60000]
loss: 1.765501 [12864/60000]
loss: 1.810394 [19264/60000]
loss: 1.695440 [25664/60000]
loss: 1.649800 [32064/60000]
loss: 1.682816 [38464/60000]
loss: 1.581166 [44864/60000]
